In [4]:
# import libraries
# import boto3, re, sys, math, json, os, sagemaker, urllib.request
# from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
# from sagemaker.predictor import csv_serializer   

import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype    
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 


In [ ]:
# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")


In [ ]:
bucket_name = 'hcp-ml-project' # <--- change this variable to a unique name for your bucket
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)


In [5]:
try:
    df = pd.read_csv('./cc_attach_project.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)


/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Success: Data loaded into dataframe.


In [6]:

to_drop = ['ORGANIZATION_ID', 'ENROLLMENT_DATE', 'DATE']
df.drop(to_drop, inplace=True, axis=1)
# df['ENROLLMENT_DATE'] = pd.to_datetime(df['ENROLLMENT_DATE'])
# df['DATE'] = pd.to_datetime(df['DATE'])
df['STAGENAME_PRODUCT'] = df['STAGENAME_PRODUCT'].fillna('not called')
df['INDUSTRY'] = df['INDUSTRY'].fillna('missing')
df['CASH_PERC'] = df['CASH_PERC'].fillna(0.0)  
df['CHECK_PERC'] = df['CHECK_PERC'].fillna(0.0)
df['OTHER_PERC'] = df['OTHER_PERC'].fillna(0.0)
df['ACH_PERC'] = df['ACH_PERC'].fillna(0.0)
df['CC_PERC'] = df['CC_PERC'].fillna(0.0)
df['BILLING_PLAN_ID'] = df['BILLING_PLAN_ID'].astype('int64')
cat_dtype = CategoricalDtype(categories=(df['PLAN_TIER'].unique()), ordered=True)
df['PLAN_TIER'] = df['PLAN_TIER'].astype(cat_dtype) #small > medium?
df['INDUSTRY'] = df['INDUSTRY'].astype('category')
df['ORGSIZE'] = df['ORGSIZE'].abs()
df['ORGSIZE'].replace(0,1, inplace=True) #should I do this?


#convert to all floats
# df = df.apply(pd.to_numeric, errors='ignore', downcast='float')

# df = pd.get_dummies(df)

In [7]:
for column in df:
    if df[column].dtype == np.float32 or df[column].dtype == np.int64:
        df[column] = df[column].astype('float64')
print(df.dtypes)

WILL_ATTACH_IN_60         float64
PLAN_TIER                category
BILLING_PLAN_ID           float64
CC_FEE                    float64
ASP                       float64
INDUSTRY                 category
ORGSIZE                   float64
BANK_CONNECTED            float64
BANK_VERIFIED             float64
CREATED                   float64
SCHEDULED                 float64
OMW                       float64
STARTED                   float64
IN_PROGRESS               float64
ESTIMATED                 float64
INVOICED                  float64
TOTALPAYMENT_COUNT        float64
TOTALPAYMENT_AMOUNT       float64
ONLINE_BOOKINGS           float64
CC_COUNT                  float64
CC_AMOUNT                 float64
INSTAPAY_AMOUNT           float64
ACH_COUNT                 float64
CASH_COUNT                float64
CASH_AMOUNT               float64
CHECK_COUNT               float64
CHECK_AMOUNT              float64
OTHER_COUNT               float64
DAYS_SINCE_ENROLLMENT     float64
REASON_TO_ADOP

In [8]:
df = pd.get_dummies(df)

In [9]:
y = df['WILL_ATTACH_IN_60']
X = df.drop('WILL_ATTACH_IN_60', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123, 
                                                    stratify=y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(964489, 169) (241123, 169) (964489,) (241123,)


In [10]:
# 5. Declare data preprocessing steps
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestClassifier(n_estimators=100, warm_start=True))
# 6. Declare hyperparameters to tune
hyperparameters = { 'randomforestclassifier__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestclassifier__max_depth': [None, 5, 3, 1]}
 
# 7. Tune model using cross-validation pipeline
clf = GridSearchCV(pipeline, hyperparameters, cv=10, refit=True,return_train_score=True)


In [11]:
X_trains, y_trains = np.array_split(X_train, indices_or_sections=20), np.array_split(y_train, indices_or_sections=20)    

for X_train_section, y_train_section in zip(X_trains, y_trains):
    print("TRAIN:", X_train_section.shape, "TEST:", y_train_section.shape)
    clf.fit(X_train_section, y_train_section)
print(clf.score(X_test,y_test))
       


TRAIN: (48225, 169) TEST: (48225,)


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to f

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to f

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to f

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to f

...

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to f

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to f

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to f

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


0.9461851420229509


In [13]:
print(clf.best_params_)
# print(clf.feature_importances_)
results = clf.cv_results_
print(clf.best_score_)
print(clf.best_estimator_)
print(results['mean_train_score'])
print(results['mean_test_score'])
joblib.dump(clf, 'rf_regressor.pkl')


{'randomforestclassifier__max_depth': None, 'randomforestclassifier__max_features': 'sqrt'}
0.9438868613138686
Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min...tors=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0, warm_start=True))])
[0.999318   0.9993203  0.99932261 0.8952504  0.89519972 0.8972365
 0.89333342 0.89367443 0.86286213 0.84227778 0.85133274 0.82589124]
[0.94382465 0.94388686 0.94262193 0.89451311 0.89420206 0.89627571
 0.89328965 0.89328965 0.86232996 0.84192518 0.85133958 0.82558477]


['rf_regressor.pkl']